# Universal Geocoder Input 
This script performs a point in polygon inclusion test. We are testing whether or not the coordinates fall within the polygons. The output file lists the IDs of polygons that pass the test. This type of transormation can also be done using a spatial join tool in ArcGIS.

This script takes GIS shapefiles as input and preprocesses them as key-value dictionaries where the key is the name of the geography and the value is a two dimensional list of vertices. This script also processes the shape data into a tableau-ready geospatial data format.

Zip code, blockgroup, and council district shapefiles can be downloaded at the county GIS portal: https://www5.kingcounty.gov/gisdataportal/
Informal neighorhoods can be downloaded at: https://data.seattle.gov/dataset/Neighborhoods/2mbt-aqqx
Urban Villages can be downloaded at: https://data.seattle.gov/dataset/Urban-Villages/ugw3-tp9e

The function geocode(float longtitude,float latitude) returns a list with the following values:

    0 - Informal Neighborhood Short Name
    1 - Informal Neighborhood Long Name
    2 - Council District
    3 - Zip Code
    4 - Urban Village
    5 - Block Group
    6 - Geographical Area

For information, contact Stephen Barham at stephen.barham@seattle.gov

In [174]:
import csv
import pandas as pd
import matplotlib.path as mplPath
import numpy as np
from matplotlib import path
from ast import literal_eval
import os
import ast

import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

import xmltodict
try:
    from urllib.request import Request, urlopen  # Python 3
except:
    from urllib2 import Request, urlopen  # Python 2
    
import pysal as ps

import xml.etree.ElementTree


In [183]:
## Create files from shapefiles. Note, this can take a long time.
# we will call shapefileToDict(directory, inputfile, nameField, typeField, conversion=False) 

directory = "V://Asset Management Program//Data Science//Geographies//"

# Blockgroup
blockGroup_Dict = shapefileToDict(directory, 'blkgrp10_shore.shp', 'GEO_ID_GRP', 'LEVEL_3', True)
w = csv.writer(open(directory + "blkgrp10_shore.csv", "w", newline=''))
for key, val in blockGroup_Dict.items():
    w.writerow([key, val])

# Urban Village
urbanVillage_Dict = shapefileToDict(directory, 'DPD_uvmfg_polygon.shp', 'UV_NAME', 'UV_TYPE', False)
w = csv.writer(open(directory + "DPD_uvmfg_polygon.csv", "w", newline=''))
for key, val in urbanVillage_Dict.items():
    w.writerow([key, val])

# Zip Code
zipcode_Dict = shapefileToDict(directory, 'zipcode.shp', 'ZIPCODE', 'COUNTY', True)
w = csv.writer(open(directory + "Zipcode.csv", "w", newline=''))
for key, val in zipcode_Dict.items():
    w.writerow([key, val])

# Informal Neighborhood ShortName
neighborhood_Dict = shapefileToDict(directory, 'Neighborhoods.shp', 'S_HOOD', 'L_HOOD', False)
w = csv.writer(open(directory + "Neighborhoods-Short.csv", "w", newline=''))
for key, val in neighborhood_Dict.items():
    w.writerow([key, val])

# Informal Neighborhood LongName
neighborhood_Dict = shapefileToDict(directory, 'Neighborhoods.shp', 'L_HOOD', 'S_HOOD', False)
w = csv.writer(open(directory + "Neighborhoods-Long.csv", "w", newline=''))
for key, val in neighborhood_Dict.items():
    w.writerow([key, val])

# Council District
councilDist_Dict = shapefileToDict(directory, 'sccdst.shp', 'SCCDST', 'NAME', True)
w = csv.writer(open(directory + "Sccdst.csv", "w", newline=''))
for key, val in councilDist_Dict.items():
    w.writerow([key, val])
df_Shapefile = ps.pdio.read_files(directory + 'sccdst.shp')



In [182]:
## Load Data
def geocode(lat,long):
    
    NeighborhoodShort = "None"
    NeighborhoodLong = "None"
    CouncilDistrict = "None"
    ZipCode = "None"
    UrbanVillage = "None"
    BlockGroup = "None"

    for key, value in csv.reader(open(directory + "Neighborhoods-Short.csv")):
        p = path.Path(ast.literal_eval(value))
        if p.contains_point((float(lat),float(long))) == True:
            NeighborhoodShort = key.split("--")[0]
            
    for key, value in csv.reader(open(directory + "Neighborhoods-Long.csv")):
        p = path.Path(ast.literal_eval(value))
        if p.contains_point((float(lat),float(long))) == True:
            NeighborhoodLong = key.split("--")[0]
            
    for key, value in csv.reader(open(directory + "sccdst.csv")):
        p = path.Path(ast.literal_eval(value))
        if p.contains_point((float(lat),float(long))) == True:
            CouncilDistrict = key.split("--")[0]
    
    for key, value in csv.reader(open(directory + "DPD_uvmfg_polygon.csv")):
        p = path.Path(ast.literal_eval(value))
        if p.contains_point((float(lat),float(long))) == True:
            UrbanVillage = key.split("--")[0]
            
    return (NeighborhoodShort, NeighborhoodLong, CouncilDistrict,UrbanVillage)

print(geocode(47.606210,-122.332071))

('Central Business District', 'DOWNTOWN', 'SCC7', 'Commercial Core')


## BUILD DICTIONARY FROM SHAPE FILES, CREATE TABLEAU MAP DATASET
Build a dictionary from the KML file. This script also creates a tableau-formatted
polygon file. Note, KML files exported from layers in ArcGIS may not work without additional processing.

In [178]:
# Build a polygons dictionary from the KML file
def shapefileToDict(directory, inputfile, nameField, typeField, conversion=False):
    
    polygons = {}
    polygonCounts = {}
    
    # read shapefile via pysal
    df_Shapefile = ps.pdio.read_files(directory + inputfile)
    
    #df_Shapefile =  df_Shapefile[df_Shapefile['GEO_ID_GRP'] == '530339901000']
    #df_Shapefile =  df_Shapefile.iloc[0:2]
    #print (df_Shapefile)
    
    # for tableau table construction
    Longitude = []
    Latitude = []
    SortOrder = []
    Location = []
    polygonCount = 0
    ID = []
    Geography = ""
    Name = ""
    Type = []
    
    for index, row in df_Shapefile.iterrows():   
        polygon = []
        polygonXY = []

        # check to make sure the name is not already in the dictionary    
        if str(row[nameField]) in polygonCounts:
            polygonCounts[str(row[nameField])] = polygonCounts[str(row[nameField])] + 1
        else:
            polygonCounts[str(row[nameField])] = 0 
            
        Name = row[nameField]
        Geography = str(row[nameField])+ "--" + str(polygonCounts[row[nameField]])
        sortOrder = 0
        geo_type = ""
        
        for vertice in (row['geometry'].vertices):
            
            coordinate = []
            status = "open"

            # WGS Coordinates - no conversion
            coordinate = (float(vertice[1]),float(vertice[0]))

            # check to see if the polygon has closed
            if coordinate in polygonXY: 
                status = "closed"
            polygonXY.append(coordinate)
                    
            # State plane coordinates - Conversion
            if conversion == True:
                coordinate = (XYtoLatLong(float(vertice[0]), float(vertice[1])))
            polygon.append(coordinate)
                           
            # for tableau
            sortOrder = sortOrder + 1
            Longitude.append(coordinate[1])
            Latitude.append(coordinate[0])
            ID.append(Geography)
            SortOrder.append(sortOrder)
            Location.append(Name)
            Type.append(row[typeField])
        
            # add vertice centroids for polygon labels in Tableau!                      
            if status == "closed": # This is a closed polygon, start a new one
                polygons[Geography] = polygon
                polygonCounts[str(row[nameField])] = polygonCounts[row[nameField]] + 1
                Geography = str(row[nameField]) + "--" + str(polygonCounts[row[nameField]])
                sortOrder = 0
                polygon = []
                polygonXY = []
                geo_type = "multi"

            else:
                geo_type = ""
                
        if geo_type != "multi":
            polygons[Geography] = polygon
    
    #Build table and export as csv        
    dfOut = pd.DataFrame()
    dfOut['id'] = ID
    dfOut['lon'] = Longitude
    dfOut['lat'] = Latitude
    dfOut['sort_Order'] = SortOrder
    dfOut['location'] = Location
    dfOut['type'] = Type
    
    #dfOut.to_csv('V:\Asset Management Program\Data Science\Geographies\Tableau' + inputfile[:-4] + '.csv', mode='w', header=True, index=False, encoding='utf-8')
    dfOut.to_csv('V://Asset Management Program//Data Science//Geographies//Tableau_' + inputfile[:-4] + '.csv', mode='w', header=True, index=False, encoding='utf-8')

    return (polygons)
    #print (dfOut)



## Point in Point Polygon Inclusion Test

In [119]:
def XYtoLatLong(X, Y):

    URL = "http://citygis/coordinateconversion/coordinateconversion.asmx/WaStatePlaneToLatLong?X=" + str(X) + "&Y=" + str(Y)
    #q = Request(URL)
    #data = urlopen(q).read()
    #return (str(data['LatLongPoint']['Latitude']), str(data['LatLongPoint']['Longitude']))
    
    tree = ET.parse(urlopen(URL))
    doc = tree.getroot()
    return (float(doc[0].text), float(doc[1].text))
    

In [161]:
URL = "http://citygis/coordinateconversion/coordinateconversion.asmx/WaStatePlaneToLatLong?X=1283237.4387780197&Y=268491.5308109835"
URL = "http://citygis/coordinateconversion/coordinateconversion.asmx/WaStatePlaneToLatLong?X=1260465.5400086045&Y=217803.2951618582"

print (URL)     

    
import  xml.etree.ElementTree as ET

tree = ET.parse(urlopen(URL))
doc = tree.getroot()
print (doc.find('LatLongPoint'))

print ((doc[0].text, doc[1].text))

#q = Request(URL)
#data = urlopen(q).read()

#print (data)
#print (str(data['LatLongPoint']['Longitude']))

http://citygis/coordinateconversion/coordinateconversion.asmx/WaStatePlaneToLatLong?X=1260465.5400086045&Y=217803.2951618582
None
('47.58668', '-122.37308')


In [5]:
directory = "V://Asset Management Program//Data Science//Geographies//"
inputfile = "blkgrp10_shore.shp"

df_Shapefile = ps.pdio.read_files(directory + inputfile)
#print (df_Shapefile)
print (df_Shapefile['geometry'][1421].area)

781868.29542933
